In [1]:
import pandas as pd
import csv
import re
from html.parser import HTMLParser
import unicodedata
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
# descripton cleaning helper functions

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def remove_email(desc):
    match = re.search(r'[\w\.-]+@[\w\.-]+', desc)
    
    if match is not None:
        email = match.group(0)
        # print(f'EMAIL={email}')
        return desc.replace(email,'')
    else:
        # print('NO EMAIL IN DESCRIPTION')
        return desc

def clean_description(html):
    #desc = unidecode.unidecode(html)
    desc = unicodedata.normalize('NFKD', html)
    
    # remove email
    desc = remove_email(desc)
    
    # replace linebreaks and paragraph tags with space
    desc = re.sub('<\\s*br\\s*>', ' ', desc)
    desc = re.sub('<\\s*br\\s*/\\s*>', ' ', desc)
    desc = re.sub('<\\s*p\\s*>', ' ', desc)
    desc = re.sub('<\\s*p\\s*/\\s*>', ' ', desc)
    
    # parse and remove other HTML tags / symbol entities
    desc = strip_tags(desc)
    
    # replace all non-alphanumeric characters
    # may or may not bite us in the ass; see how it goes
    desc = re.sub('[^a-zA-Z0-9]', ' ', desc)
    
    # lowercase result string
    desc = desc.lower()
    
    # return cleaned string with extra spaces removed
    return ' '.join(desc.split())   
    

In [3]:
# term frequency and tf-idf functions (test)
def setTermFrequency(index, desc, orig):
    # create new vectorizer for each description
    vectorizer = CountVectorizer(strip_accents='unicode', stop_words='english', ngram_range=(1,1))
    """
    try:
        trans = vectorizer.fit_transform([desc])
    except:
        print('INDEX = ' + str(index))
        print('DESCRIPTION = ' + orig)
        print('ISEMPTY = ' + str(orig == ''))
        print('CLEANED = ' + desc)
        print('ISCLEANEMPTY = ' + str(desc == ''))
    """
    # need to call this before getting vocab
    trans = vectorizer.fit_transform([desc])
    vectorizer._validate_vocabulary()
    feat_names = vectorizer.get_feature_names()
    feat_count = trans.toarray()[0]
    orig_data.at[index, 'tf'] = [feat_names, feat_count]
    #return feat_names#, feat_count

In [4]:
# term frequency and tf-idf functions
def setTermFrequency(index, desc):
    # get count vectorizer
    count_vectorizer = CountVectorizer(strip_accents='unicode', stop_words='english', ngram_range=(1,1))
    desc_count = count_vectorizer.fit_transform([desc])
    
    # get tf transformer
    tf_transformer = TfidfTransformer(use_idf=False)
    desc_tf = tf_transformer.fit_transform(desc_count)
    
    # get tfidf transformer
    tfidf_transformer = TfidfTransformer(use_idf=True)
    desc_tfidf = tfidf_transformer.fit_transform(desc_count)
    
    orig_data.at[index, 'tf'] = desc_tf
    orig_data.at[index, 'tfidf'] = desc_tfidf

In [5]:
# misc properties before running code

# increases text displayed in pandas table for Jupyter Notebook
pd.options.display.max_colwidth = 10000

In [6]:
# read csv into pandas DataFrame (sample.csv = 100000)
orig_data = pd.read_csv('sample.csv', keep_default_na=False)

# remove listings where description is blank (100000 -> 99680)
orig_data = orig_data[orig_data['description'] != '']

In [7]:
# clean descriptions (takes some time) and remove blanks after cleaning (99680 -> 99641)
orig_data['cleaned'] = orig_data['description'].apply(clean_description)
orig_data = orig_data[orig_data['cleaned'] != '']

In [8]:
# CountVectorizer expects a list/iterable, so change string into a one-element list
# input = [orig_data.loc[0,'description']]
# vectorizer = CountVectorizer(strip_accents='unicode', stop_words='english', ngram_range=(1,1))
# foo = vectorizer.fit_transform(input)
# vectorizer._validate_vocabulary()
# # gets unique words
# bar = vectorizer.get_feature_names()
# # gets list of list of count of each word (1 list of list with 305 numbers)
# baz = foo.toarray()
# baz = baz[0]
# print(bar)
# print(len(bar))
# print(baz)
# print(len(baz))
# print(foo)

In [9]:
# add term frequency and tf-idf columns and values
orig_data = orig_data.assign(tf = pd.Series(dtype=object))
orig_data = orig_data.assign(tfidf = pd.Series(dtype=object))

for index, row in orig_data.iterrows():
    #setTermFrequency(index, row['cleaned'], row['description'])
    setTermFrequency(index, row['cleaned'])

In [10]:
#orig_data.iloc[0,4].toarray()
orig_data.head()

title  \
0                           Field Support Rep I   
1                         Mortgage Loan Officer   
2  Analyst  Advanced Analytics (Firm Internal)   
3                   Taco Bell Assistant Manager   
4            CDL-A Owner Operator Truck Drivers   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [16]:
print(len(orig_data.index))
# 70:30 train-test split; 99641 * 0.7 ~ 69750
df_train = orig_data[0:69750]
df_test = orig_data[69750:]
print(len(df_train.index))
print(len(df_test.index))
df_train.head()

99641
69750
29891


title  \
0                           Field Support Rep I   
1                         Mortgage Loan Officer   
2  Analyst  Advanced Analytics (Firm Internal)   
3                   Taco Bell Assistant Manager   
4            CDL-A Owner Operator Truck Drivers   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       